In [ ]:
# imports
import os
import json
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import numpy as np
import pandas as pd
import nbconvert
import nbformat
import zipfile
import csv
import time
import ast
# jenna

In [ ]:
# connect to local elastic search host
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST], timeout=30)

In [ ]:
# load jenna python file
%run jennacode.py

In [ ]:
# Initialise the elastic search index
init_code_index = {
    "settings" : {
        "number_of_shards": 5,
        "number_of_replicas": 1,
    },
    'mappings': {
            'properties': {
                'nb_id': {'type': 'integer'},
                'language': {'type': 'keyword'},
                'code': {'type': 'text'},
                'imports': {'type': 'text'},
                'functions': {'type': 'text'},
                'classes': {'type': 'text'},
        }
    }
}
es.indices.create(index = 'code', body = init_code_index)

In [ ]:
# Initialise the elastic search index
init_comments_index = {
    "settings" : {
        "number_of_shards": 5,
        "number_of_replicas": 1,
        "analysis": {
            "analyzer": {
                "text_analyzer": {
                    "type": "standard",
                    "stopwords": "_english_"
                }
            }
        }
    },
    'mappings': {
            'properties': {
                'nb_id': {'type': 'integer'},
                'comments': {'type': 'text', 
                             "analyzer":"text_analyzer"
                            },
        }
    }
}
es.indices.create(index = 'comments2', body = init_comments_index)

In [ ]:
# tes_df1 = pd.read_pickle('df_bb2733859v_2_1_t.pkl') # DONE
# tes_df2 = pd.read_pickle('bb2733859v_3_1_new.pkl') # DONE
# tes_df3 = pd.read_pickle('bb2733859v_4_1(1)_new.pkl') # DONE>
# tes_df4 = pd.read_pickle('bb2733859v_5_1(1)_new.pkl') # DONE>
#tes_df5 = pd.read_pickle('bb2733859v_6_1(1)_new.pkl') # Done
tes_df = pd.read_pickle('bb2733859v_7_1(1)_new.pkl') #
# tes_df = pd.concat([tes_df1,tes_df2])
# print(len(tes_df1),len(tes_df2),len(tes_df))
# tes_df = pd.concat([tes_df3,tes_df4])
# print(len(tes_df3),len(tes_df4),len(tes_df))

In [ ]:
es_df = tes_df
all_info = []
for i, row in es_df.iterrows():
    cell_info = {
        "file": row['name'],
        "cell_id": row['nb_id'],
        "language": row['language'],
        "lines_of_code": 0,
        "code": row['code'],
        "imports": [],
        "parsed_ast": False,
        "num_functions": 0,
        "functions": [],
        "num_classes": 0,
        "classes": [],
        "comments": [],
        "num_comments": 0,
    }
    if row['language'] == 'python':
        cell_info = parse_py(cell_info)
    if row['language'] == 'r':
        cell_info = parse_r(cell_info)
    if row['language'] == 'julia':
        cell_info = parse_ju(cell_info)
    all_info.append(cell_info)
parsed_df = pd.DataFrame(all_info)

In [ ]:
parsed_df.head()

In [ ]:
len(tes_df)

In [ ]:
es_df = tes_df
def gen_code(es_df):
    for i, row in es_df.iterrows():
        t = {
            "_id": row['cell_id'],
            "language": row['language'],
            'code': row['code'],
            'imports': row['imports'],
            'functions': row['functions'],
            'classes': row['classes'],
            }
        yield t

errors = []
for ok, action in helpers.parallel_bulk(client=es, index='code', actions=gen_code(parsed_df)):
    if not ok:
        errors.append(action)

In [ ]:
es_df = tes_df
def gen_md(es_df):
    for i, row in es_df.iterrows():
        t = {
            "_id": row['nb_id'],
            'comments': row['comments']
            }
        yield t

errors = []
for ok, action in helpers.parallel_bulk(client=es, index='comments2', actions=gen_md(es_df)):
    if not ok:
        errors.append(action)